In [ ]:
pip install --upgrade pip

In [ ]:
%pip install -q safetensors

In [ ]:
import pandas as pd
import os
import torch.nn.functional as F
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [ ]:
#%pip install -q safetensors

import torch
from PIL import Image
from transformers import CLIPProcessor, CLIPModel

# 1) Load model & processor
model_id = "openai/clip-vit-base-patch32"

# Force safetensors to avoid torch.load() (and the torch>=2.6 restriction / CVE gate)
model = CLIPModel.from_pretrained(model_id, use_safetensors=True)
processor = CLIPProcessor.from_pretrained(model_id)

# 2) Choose device: GPU if available, else CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

if device.type == "cuda":
    # Print GPU name and count if on CUDA
    print(f"Number of GPUs available: {torch.cuda.device_count()}")
    for i in range(torch.cuda.device_count()):
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")

# 3) Move model to that device
model.to(device)

# 4) Load and preprocess an image
image_path = "Images\\18759-guard301.jpg"
image = Image.open(image_path)

# texts = ["cat", "dog", "car", "fruit bowl", "city", "flower"]
texts = ["Fransesco Guardi", "Picasso", "Van Gogh", "Leonardo da Vinci",
         "Claude Monet", "Salvador Dali", "Rembrandt", "Michelangelo", "Raphael",
         "Caravaggio", "Titian", "El Greco", "Goya", "Vermeer", "Botticelli"]

# 5) Preprocess inputs (still on CPU now)
inputs = processor(text=texts, images=image, return_tensors="pt", padding=True)

# 6) Move all input tensors to the same device as the model
inputs = {k: v.to(device) for k, v in inputs.items()}

# 7) Run the model
outputs = model(**inputs)
image_embeddings = outputs.image_embeds
text_embeddings  = outputs.text_embeds

# 8) Compute similarity
import torch.nn.functional as F
similarity_scores = F.cosine_similarity(image_embeddings, text_embeddings)

print("Similarity scores:", similarity_scores)

best_idx = similarity_scores.argmax().item()
print(f"Best match: '{texts[best_idx]}' with score {similarity_scores[best_idx]:.4f}")


## Analysis: Authors with Paintings from Different Time Frames

Let's check if the same author has paintings made in different time frames in the semart_train.csv dataset.

In [ ]:
# Load the semart_train.csv file
df_train = pd.read_csv("semart_full_cleaned.csv")

# Display basic info about the dataset
print(f"Total number of paintings: {len(df_train)}")
print(f"\nColumn names: {df_train.columns.tolist()}")
print(f"\nFirst few rows:")
df_train.head()

## Author Prediction Using CLIP

Using the same CLIP similarity logic, let's predict the author of 10 random paintings by comparing each image against all authors in the dataset.

In [ ]:
# Get all unique authors from the dataset
all_authors = df_train['AUTHOR'].unique().tolist()
print(f"Total number of unique authors in dataset: {len(all_authors)}")

# Select 10 random paintings from the dataset
import random
random.seed(42)  # For reproducibility
sample_paintings = df_train.sample(n=10, random_state=42)

print(f"\nSelected {len(sample_paintings)} random paintings for prediction:")
sample_paintings[['IMAGE_FILE', 'AUTHOR', 'TITLE', 'DATE']].head(10)

In [ ]:
# Function to predict author for a given image
def predict_author_for_image(image_path, authors_list, model, processor, device):
    """
    Predict the author of a painting using CLIP similarity
    """
    try:
        # Load and preprocess the image
        image = Image.open(image_path)
        
        # Preprocess inputs
        inputs = processor(text=authors_list, images=image, return_tensors="pt", padding=True)
        
        # Move all input tensors to the same device as the model
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Run the model
        outputs = model(**inputs)
        image_embeddings = outputs.image_embeds
        text_embeddings = outputs.text_embeds
        
        # Compute similarity
        similarity_scores = F.cosine_similarity(image_embeddings, text_embeddings)
        
        # Get best match
        best_idx = similarity_scores.argmax().item()
        best_author = authors_list[best_idx]
        best_score = similarity_scores[best_idx].item()
        
        # Get top 5 matches
        top5_indices = similarity_scores.argsort(descending=True)[:5]
        top5_matches = [(authors_list[idx], similarity_scores[idx].item()) for idx in top5_indices]
        
        return best_author, best_score, top5_matches
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None, None, None

print("Function defined successfully!")

## Author Prediction for Top 100 Most Prolific Artists

Let's improve prediction accuracy by focusing on the 100 authors with the most paintings in the dataset (excluding Unknown artists).

In [ ]:
# Find the top 100 authors with the most paintings (excluding Unknown artists)
author_counts = df_train['AUTHOR'].value_counts()

# Filter out authors with "Unknown" in their name
author_counts_filtered = author_counts[~author_counts.index.str.contains('Unknown', case=False, na=False)]
top_100_authors = author_counts_filtered.head(100).index.tolist()

print(f"Top 100 authors with most paintings (excluding Unknown artists):")
print("="*80)
for i, author in enumerate(top_100_authors, 1):
    count = author_counts[author]
    print(f"{i:3d}. {author:40s} - {count:3d} paintings")

In [ ]:
# Use ALL paintings from the top 100 authors
paintings_top100 = df_train[df_train['AUTHOR'].isin(top_100_authors)]

print(f"Total paintings by top 100 authors: {len(paintings_top100)}")
print(f"\nWill predict authors for all {len(paintings_top100)} paintings")
print(f"\nFirst 10 paintings:")
paintings_top100[['IMAGE_FILE', 'AUTHOR', 'TITLE', 'DATE']].head(10)

In [ ]:
# Predict authors for ALL paintings by top 100 authors (using only top 100 authors as candidates)
top100_results = []

print(f"Predicting authors for all {len(paintings_top100)} paintings (Top 100 authors only)...\n")
print("="*100)

total_paintings = len(paintings_top100)
for counter, (idx, row) in enumerate(paintings_top100.iterrows(), 1):
    
    image_file = row['IMAGE_FILE']
    true_author = row['AUTHOR']
    title = row['TITLE']
    image_path = os.path.join("Images", image_file)
    
    
    # Show progress every 100 paintings or for first 10
    if counter <= 10 or counter % 100 == 0:
        print(f"\n[{counter}/{total_paintings}] Processing: {title} by {true_author}")
    
    # Predict author using only top 100 authors
    predicted_author, confidence, top5_matches = predict_author_for_image(
        image_path, top_100_authors, model, processor, device
    )
    
    if predicted_author:
        is_correct = predicted_author == true_author
        symbol = "✅" if is_correct else "❌"
        
        print(f"   {symbol} Predicted Author: {predicted_author} (confidence: {confidence:.4f})")
        # Only show detailed predictions for first 10 or every 100th painting
        if counter <= 10 or counter % 100 == 0:
            print(f"   {symbol} Predicted: {predicted_author} (confidence: {confidence:.4f})")
            if not is_correct:
                print(f"      Top 5: {', '.join([f'{a} ({s:.3f})' for a, s in top5_matches[:5]])}")
        top100_results.append({
            'image_file': image_file,
            'title': title,
            'true_author': true_author,
            'predicted_author': predicted_author,
            'confidence': confidence,
            'correct': is_correct
        })
    

print(f"\n\nPrediction complete!")

In [ ]:
# Summary of results for top 100 authors
if top100_results:
    
    top100_results_df = pd.DataFrame(top100_results)
    
    print("\n" + "="*100)
    print("SUMMARY OF PREDICTIONS (TOP 100 AUTHORS)")
    print("="*100)
    
    # Basic accuracy
    accuracy = top100_results_df['correct'].sum() / len(top100_results_df) * 100
    print(f"\nTotal paintings processed: {len(top100_results_df)}")
    print(f"Accuracy: {accuracy:.2f}% ({top100_results_df['correct'].sum()}/{len(top100_results_df)} correct predictions)")
    
    # Calculate precision, recall, and F1 score
    y_true = top100_results_df['true_author']
    y_pred = top100_results_df['predicted_author']
    
    precision_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
    recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
    f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)
    
    precision_weighted = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall_weighted = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    
    print(f"\nEvaluation Metrics:")
    print(f"  Macro Average:")
    print(f"    Precision: {precision_macro:.4f}")
    print(f"    Recall:    {recall_macro:.4f}")
    print(f"    F1 Score:  {f1_macro:.4f}")
    print(f"  Weighted Average:")
    print(f"    Precision: {precision_weighted:.4f}")
    print(f"    Recall:    {recall_weighted:.4f}")
    print(f"    F1 Score:  {f1_weighted:.4f}")
    
    print(f"\nConfidence Statistics:")
    print(f"  Average: {top100_results_df['confidence'].mean():.4f}")
    print(f"  Median: {top100_results_df['confidence'].median():.4f}")
    print(f"  Max: {top100_results_df['confidence'].max():.4f}")
    print(f"  Min: {top100_results_df['confidence'].min():.4f}")
    
    # Accuracy by confidence level
    correct_df = top100_results_df[top100_results_df['correct']]
    incorrect_df = top100_results_df[~top100_results_df['correct']]
    
    print(f"\nCorrect predictions by confidence:")
    print(f"  Correct predictions avg confidence: {correct_df['confidence'].mean():.4f}")
    print(f"  Incorrect predictions avg confidence: {incorrect_df['confidence'].mean():.4f}")
    
    # Per-author accuracy (for authors with at least 10 paintings)
    print(f"\nPer-Author Performance (authors with 10+ paintings):")
    author_stats = []
    for author in top100_results_df['true_author'].unique():
        author_data = top100_results_df[top100_results_df['true_author'] == author]
        if len(author_data) >= 10:
            author_accuracy = (author_data['correct'].sum() / len(author_data)) * 100
            author_stats.append({
                'author': author,
                'total': len(author_data),
                'correct': author_data['correct'].sum(),
                'accuracy': author_accuracy
            })
    
    author_stats_df = pd.DataFrame(author_stats).sort_values('accuracy', ascending=False)
    print(f"\nTop 10 Best Performing Authors:")
    for idx, row in author_stats_df.head(10).iterrows():
        print(f"  {row['author']:40s} - {row['accuracy']:.1f}% ({row['correct']}/{row['total']})")
    
    print(f"\nBottom 10 Authors:")
    for idx, row in author_stats_df.tail(10).iterrows():
        print(f"  {row['author']:40s} - {row['accuracy']:.1f}% ({row['correct']}/{row['total']})")
    
    # Most commonly confused authors
    print(f"\nMost Common Prediction Errors (Top 10):")
    errors = top100_results_df[~top100_results_df['correct']]
    
    if len(errors) > 0:
        error_pairs = errors.groupby(['true_author', 'predicted_author']).size().sort_values(ascending=False).head(10)
        for (true_a, pred_a), count in error_pairs.items():
            print(f"  {true_a} → {pred_a}: {count} times")
    
    print("\n\nDetailed Results (first 50 rows):")
    display(top100_results_df[['title', 'true_author', 'predicted_author', 'confidence', 'correct']].head(50))
    
    print("\nLast 20 rows:")
    display(top100_results_df[['title', 'true_author', 'predicted_author', 'confidence', 'correct']].tail(20))
else:
    print("No results to display.")

In [ ]:
top100_results_df.to_csv("top100_results_df.csv", index=False)
author_stats_df.to_csv("author_stats_df.csv", index=False)

## Apply Grayscale Transformation Using PyTorch

Let's select a random painting and apply grayscale transformation using PyTorch.

In [ ]:
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
from PIL import Image
import torch

# Select a random painting from the dataset
random_painting = df_train.sample(n=1, random_state=42)
image_file = random_painting['IMAGE_FILE'].values[0]
title = random_painting['TITLE'].values[0]
author = random_painting['AUTHOR'].values[0]
image_path = os.path.join("Images", image_file)

print(f"Selected painting: {title}")
print(f"Author: {author}")
print(f"File: {image_file}")

# Load the image
original_image = Image.open(image_path)

# Convert PIL Image to PyTorch tensor
to_tensor = transforms.ToTensor()
image_tensor = to_tensor(original_image)

print(f"\nOriginal image shape: {image_tensor.shape}")  # Should be [C, H, W]
print(f"Image size: {original_image.size}")

# Apply grayscale transformation using PyTorch
# Method 1: Using torchvision transforms
grayscale_transform = transforms.Grayscale(num_output_channels=1)
grayscale_tensor = grayscale_transform(original_image)
grayscale_tensor = to_tensor(grayscale_tensor)

print(f"Grayscale tensor shape: {grayscale_tensor.shape}")  # Should be [1, H, W]

# Convert tensors back to images for display
to_pil = transforms.ToPILImage()
grayscale_image = to_pil(grayscale_tensor)

# Display original and grayscale images side by side
fig, axes = plt.subplots(1, 2, figsize=(15, 7))

axes[0].imshow(original_image)
axes[0].set_title(f'Original\n{title}\nby {author}', fontsize=10)
axes[0].axis('off')

axes[1].imshow(grayscale_image, cmap='gray')
axes[1].set_title(f'Grayscale (PyTorch)\n{title}\nby {author}', fontsize=10)
axes[1].axis('off')

plt.tight_layout()
plt.show()

print("\nGrayscale transformation applied successfully using PyTorch!")

## Re-predict Authors on Grayscale Images

Let's take all the paintings where the author was correctly predicted and apply grayscale transformation to them, then re-predict to see how grayscale affects prediction accuracy.

In [ ]:
# Function to predict author for a grayscale image
def predict_author_for_grayscale_image(image_path, authors_list, model, processor, device):
    """
    Apply grayscale transformation and predict the author of a painting using CLIP similarity
    """
    try:
        # Load the image
        image = Image.open(image_path)
        
        # Apply grayscale transformation
        grayscale_transform = transforms.Grayscale(num_output_channels=3)  # Convert to 3 channels for CLIP
        grayscale_image = grayscale_transform(image)
        
        # Preprocess inputs
        inputs = processor(text=authors_list, images=grayscale_image, return_tensors="pt", padding=True)
        
        # Move all input tensors to the same device as the model
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Run the model
        outputs = model(**inputs)
        image_embeddings = outputs.image_embeds
        text_embeddings = outputs.text_embeds
        
        # Compute similarity
        similarity_scores = F.cosine_similarity(image_embeddings, text_embeddings)
        
        # Get best match
        best_idx = similarity_scores.argmax().item()
        best_author = authors_list[best_idx]
        best_score = similarity_scores[best_idx].item()
        
        return best_author, best_score
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None, None

print("Grayscale prediction function defined successfully!")

In [ ]:
# Filter for correctly predicted paintings
correct_predictions = [result for result in top100_results if result['correct']]

print(f"Total correct predictions (original color images): {len(correct_predictions)}")
print(f"We will apply grayscale to these {len(correct_predictions)} images and re-predict")

# Show sample of correctly predicted paintings
print("\nSample of correctly predicted paintings:")
for i, result in enumerate(correct_predictions[:10], 1):
    print(f"{i}. {result['title']} by {result['true_author']} (confidence: {result['confidence']:.4f})")

In [ ]:
# Re-predict authors on grayscale versions of correctly predicted images
grayscale_results = []

print(f"Re-predicting authors on {len(correct_predictions)} grayscale images...\n")
print("="*100)

total = len(correct_predictions)
for counter, result in enumerate(correct_predictions, 1):
    
    image_file = result['image_file']
    true_author = result['true_author']
    original_prediction = result['predicted_author']
    original_confidence = result['confidence']
    title = result['title']
    image_path = os.path.join("Images", image_file)
    
    # Show progress every 200 paintings or for first 10
    if counter <= 10 or counter % 200 == 0:
        print(f"\n[{counter}/{total}] Processing: {title} by {true_author}")
    
    # Predict author on grayscale image
    predicted_author, confidence = predict_author_for_grayscale_image(
        image_path, top_100_authors, model, processor, device
    )
    
    if predicted_author:
        is_correct = predicted_author == true_author
        symbol = "✅" if is_correct else "❌"
        
        # Only show detailed predictions for first 10 or every 200th painting
        if counter <= 10 or counter % 200 == 0:
            print(f"   Original (color): ✅ {original_prediction} (confidence: {original_confidence:.4f})")
            print(f"   Grayscale: {symbol} {predicted_author} (confidence: {confidence:.4f})")
        
        grayscale_results.append({
            'image_file': image_file,
            'title': title,
            'true_author': true_author,
            'original_prediction': original_prediction,
            'original_confidence': original_confidence,
            'grayscale_prediction': predicted_author,
            'grayscale_confidence': confidence,
            'still_correct': is_correct
        })

print(f"\n\nGrayscale re-prediction complete!")

In [ ]:
# Comprehensive comparison of color vs grayscale predictions
if grayscale_results:
    grayscale_results_df = pd.DataFrame(grayscale_results)
    
    print("\n" + "="*100)
    print("GRAYSCALE vs COLOR PREDICTION COMPARISON")
    print("="*100)
    
    # Calculate accuracy on grayscale
    grayscale_accuracy = grayscale_results_df['still_correct'].sum() / len(grayscale_results_df) * 100
    
    print(f"\nOriginal Color Images:")
    print(f"  Total paintings tested: {len(grayscale_results_df)}")
    print(f"  All were correctly predicted: 100% (these were the correct predictions from original test)")
    
    print(f"\nGrayscale Images:")
    print(f"  Total paintings tested: {len(grayscale_results_df)}")
    print(f"  Still correctly predicted: {grayscale_results_df['still_correct'].sum()}")
    print(f"  Accuracy: {grayscale_accuracy:.2f}%")
    
    # Calculate drop in accuracy
    accuracy_drop = 100 - grayscale_accuracy
    print(f"\nAccuracy Drop: {accuracy_drop:.2f}%")
    print(f"  ({grayscale_results_df['still_correct'].sum()}/{len(grayscale_results_df)} paintings remained correct after grayscale)")
    
    # Confidence comparison
    print(f"\nConfidence Statistics:")
    print(f"  Original (color) - Average: {grayscale_results_df['original_confidence'].mean():.4f}")
    print(f"  Grayscale - Average: {grayscale_results_df['grayscale_confidence'].mean():.4f}")
    
    confidence_diff = grayscale_results_df['grayscale_confidence'].mean() - grayscale_results_df['original_confidence'].mean()
    print(f"  Confidence change: {confidence_diff:+.4f}")
    
    # Analyze paintings that became incorrect
    became_incorrect = grayscale_results_df[~grayscale_results_df['still_correct']]
    
    if len(became_incorrect) > 0:
        print(f"\n\nPaintings that became INCORRECT after grayscale ({len(became_incorrect)} total):")
        print("-"*100)
        
        print(f"\nMost common new predictions (what model confused them with):")
        new_predictions = became_incorrect['grayscale_prediction'].value_counts().head(10)
        for author, count in new_predictions.items():
            print(f"  {author}: {count} paintings")
        
        print(f"\nAuthors most affected by grayscale (lost correct predictions):")
        affected_authors = became_incorrect['true_author'].value_counts().head(10)
        for author, count in affected_authors.items():
            total_by_author = len(grayscale_results_df[grayscale_results_df['true_author'] == author])
            print(f"  {author}: {count}/{total_by_author} became incorrect ({count/total_by_author*100:.1f}%)")
        
        print(f"\nSample of paintings that lost correct prediction:")
        display(became_incorrect[['title', 'true_author', 'original_prediction', 'grayscale_prediction', 
                                   'original_confidence', 'grayscale_confidence']].head(20))
    
    # Paintings that remained correct
    remained_correct = grayscale_results_df[grayscale_results_df['still_correct']]
    
    if len(remained_correct) > 0:
        print(f"\n\nPaintings that remained CORRECT after grayscale ({len(remained_correct)} total):")
        print("-"*100)
        
        print(f"\nAuthors most robust to grayscale:")
        robust_authors = remained_correct['true_author'].value_counts().head(10)
        for author, count in robust_authors.items():
            total_by_author = len(grayscale_results_df[grayscale_results_df['true_author'] == author])
            print(f"  {author}: {count}/{total_by_author} remained correct ({count/total_by_author*100:.1f}%)")
        
        print(f"\nSample of paintings that remained correct:")
        display(remained_correct[['title', 'true_author', 'original_confidence', 'grayscale_confidence']].head(20))
    
    print("\n\nAll Results (first 50 rows):")
    display(grayscale_results_df[['title', 'true_author', 'original_prediction', 'grayscale_prediction', 
                                   'original_confidence', 'grayscale_confidence', 'still_correct']].head(50))

else:
    print("No results to display.")

In [ ]:
import matplotlib.pyplot as plt
from PIL import Image
import os

from torchvision.transforms import v2

# Select a random painting from the dataset
random_painting = df_train.sample(n=1, random_state=42)
image_file = random_painting['IMAGE_FILE'].values[0]
title = random_painting['TITLE'].values[0]
author = random_painting['AUTHOR'].values[0]
image_path = os.path.join("Images", image_file)

print(f"Selected painting: {title}")
print(f"Author: {author}")
print(f"File: {image_file}")

# Load the image (IMPORTANT: convert to RGB)
original_image = Image.open(image_path).convert("RGB")

print(f"\nOriginal image size: {original_image.size}")

# Define RandomPerspective transform
perspective_transform = v2.RandomPerspective(
    distortion_scale=0.6,
    p=1.0
)

# Generate multiple perspective-transformed images
num_variants = 4
perspective_images = [
    perspective_transform(original_image) for _ in range(num_variants)
]

# Plot original + perspective variants
fig, axes = plt.subplots(1, num_variants + 1, figsize=(18, 5))

# Original
axes[0].imshow(original_image)
axes[0].set_title(f"Original\n{title}\nby {author}", fontsize=10)
axes[0].axis("off")

# Perspective variants
for i, img in enumerate(perspective_images, start=1):
    axes[i].imshow(img)
    axes[i].set_title(f"RandomPerspective #{i}", fontsize=10)
    axes[i].axis("off")

plt.tight_layout()
plt.show()

print("\nRandomPerspective transformation applied successfully!")


In [ ]:
def predict_author_for_perspective_image(
    image_path,
    authors_list,
    model,
    processor,
    device,
    distortion_scale=0.6
):
    """
    Apply RandomPerspective transform and predict author using CLIP
    """
    try:
        # Load image
        image = Image.open(image_path).convert("RGB")

        # Random perspective transform
        perspective_transform = v2.RandomPerspective(
            distortion_scale=distortion_scale,
            p=1.0
        )

        transformed_image = perspective_transform(image)

        # CLIP preprocessing
        inputs = processor(
            text=authors_list,
            images=transformed_image,
            return_tensors="pt",
            padding=True
        )

        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model(**inputs)
            image_embeds = outputs.image_embeds
            text_embeds = outputs.text_embeds

            similarity_scores = F.cosine_similarity(image_embeds, text_embeds)

        best_idx = similarity_scores.argmax().item()
        best_author = authors_list[best_idx]
        best_score = similarity_scores[best_idx].item()

        return best_author, best_score

    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None, None


In [ ]:
perspective_results = []

print(f"Re-predicting authors with RandomPerspective on {len(correct_predictions)} images...\n")
print("=" * 100)

total = len(correct_predictions)

for counter, result in enumerate(correct_predictions, 1):

    image_file = result['image_file']
    true_author = result['true_author']
    original_prediction = result['predicted_author']
    original_confidence = result['confidence']
    title = result['title']
    image_path = os.path.join("Images", image_file)

    if counter <= 10 or counter % 200 == 0:
        print(f"\n[{counter}/{total}] Processing: {title} by {true_author}")

    predicted_author, confidence = predict_author_for_perspective_image(
        image_path,
        top_100_authors,
        model,
        processor,
        device,
        distortion_scale=0.6
    )

    if predicted_author:
        is_correct = predicted_author == true_author
        symbol = "✅" if is_correct else "❌"

        if counter <= 10 or counter % 200 == 0:
            print(f"   Original (color): ✅ {original_prediction} ({original_confidence:.4f})")
            print(f"   Perspective: {symbol} {predicted_author} ({confidence:.4f})")

        perspective_results.append({
            'image_file': image_file,
            'title': title,
            'true_author': true_author,
            'original_prediction': original_prediction,
            'original_confidence': original_confidence,
            'perspective_prediction': predicted_author,
            'perspective_confidence': confidence,
            'still_correct': is_correct
        })

print("\n\nRandomPerspective re-prediction complete!")


In [ ]:
perspective_df = pd.DataFrame(perspective_results)

perspective_accuracy = (
    perspective_df['still_correct'].sum() / len(perspective_df) * 100
)

print(f"\nRandomPerspective Accuracy: {perspective_accuracy:.2f}%")
print(f"Accuracy Drop: {100 - perspective_accuracy:.2f}%")

print(f"\nConfidence:")
print(f"  Original avg: {perspective_df['original_confidence'].mean():.4f}")
print(f"  Perspective avg: {perspective_df['perspective_confidence'].mean():.4f}")


In [ ]:
# Comprehensive comparison of color vs RandomPerspective predictions
if perspective_results:
    perspective_results_df = pd.DataFrame(perspective_results)
    
    print("\n" + "="*100)
    print("RANDOM PERSPECTIVE vs COLOR PREDICTION COMPARISON")
    print("="*100)
    
    # Calculate accuracy on perspective images
    perspective_accuracy = (
        perspective_results_df['still_correct'].sum()
        / len(perspective_results_df) * 100
    )
    
    print(f"\nOriginal Color Images:")
    print(f"  Total paintings tested: {len(perspective_results_df)}")
    print(f"  All were correctly predicted: 100% (these were the correct predictions from original test)")
    
    print(f"\nRandomPerspective Images:")
    print(f"  Total paintings tested: {len(perspective_results_df)}")
    print(f"  Still correctly predicted: {perspective_results_df['still_correct'].sum()}")
    print(f"  Accuracy: {perspective_accuracy:.2f}%")
    
    # Calculate drop in accuracy
    accuracy_drop = 100 - perspective_accuracy
    print(f"\nAccuracy Drop: {accuracy_drop:.2f}%")
    print(
        f"  ({perspective_results_df['still_correct'].sum()}"
        f"/{len(perspective_results_df)} paintings remained correct after RandomPerspective)"
    )
    
    # Confidence comparison
    print(f"\nConfidence Statistics:")
    print(f"  Original (color) - Average: {perspective_results_df['original_confidence'].mean():.4f}")
    print(f"  RandomPerspective - Average: {perspective_results_df['perspective_confidence'].mean():.4f}")
    
    confidence_diff = (
        perspective_results_df['perspective_confidence'].mean()
        - perspective_results_df['original_confidence'].mean()
    )
    print(f"  Confidence change: {confidence_diff:+.4f}")
    
    # Analyze paintings that became incorrect
    became_incorrect = perspective_results_df[~perspective_results_df['still_correct']]
    
    if len(became_incorrect) > 0:
        print(
            f"\n\nPaintings that became INCORRECT after RandomPerspective "
            f"({len(became_incorrect)} total):"
        )
        print("-"*100)
        
        print(f"\nMost common new predictions (what model confused them with):")
        new_predictions = (
            became_incorrect['perspective_prediction']
            .value_counts()
            .head(10)
        )
        for author, count in new_predictions.items():
            print(f"  {author}: {count} paintings")
        
        print(f"\nAuthors most affected by RandomPerspective (lost correct predictions):")
        affected_authors = became_incorrect['true_author'].value_counts().head(10)
        for author, count in affected_authors.items():
            total_by_author = len(
                perspective_results_df[
                    perspective_results_df['true_author'] == author
                ]
            )
            print(
                f"  {author}: {count}/{total_by_author} became incorrect "
                f"({count/total_by_author*100:.1f}%)"
            )
        
        print(f"\nSample of paintings that lost correct prediction:")
        display(
            became_incorrect[
                [
                    'title',
                    'true_author',
                    'original_prediction',
                    'perspective_prediction',
                    'original_confidence',
                    'perspective_confidence'
                ]
            ].head(20)
        )
    
    # Paintings that remained correct
    remained_correct = perspective_results_df[perspective_results_df['still_correct']]
    
    if len(remained_correct) > 0:
        print(
            f"\n\nPaintings that remained CORRECT after RandomPerspective "
            f"({len(remained_correct)} total):"
        )
        print("-"*100)
        
        print(f"\nAuthors most robust to RandomPerspective:")
        robust_authors = remained_correct['true_author'].value_counts().head(10)
        for author, count in robust_authors.items():
            total_by_author = len(
                perspective_results_df[
                    perspective_results_df['true_author'] == author
                ]
            )
            print(
                f"  {author}: {count}/{total_by_author} remained correct "
                f"({count/total_by_author*100:.1f}%)"
            )
        
        print(f"\nSample of paintings that remained correct:")
        display(
            remained_correct[
                [
                    'title',
                    'true_author',
                    'original_confidence',
                    'perspective_confidence'
                ]
            ].head(20)
        )
    
    print("\n\nAll Results (first 50 rows):")
    display(
        perspective_results_df[
            [
                'title',
                'true_author',
                'original_prediction',
                'perspective_prediction',
                'original_confidence',
                'perspective_confidence',
                'still_correct'
            ]
        ].head(50)
    )

else:
    print("No results to display.")


In [ ]:
# Function to predict author for an elastic-transformed image
def predict_author_for_elastic_image(image_path, authors_list, model, processor, device, alpha=250.0, sigma=5.0):
    """
    Apply elastic transformation and predict the author of a painting using CLIP similarity
    """
    try:
        # Load the image
        image = Image.open(image_path)
        
        # Apply elastic transformation
        elastic_transform = transforms.ElasticTransform(alpha=alpha, sigma=sigma)
        elastic_image = elastic_transform(image)
        
        # Preprocess inputs
        inputs = processor(text=authors_list, images=elastic_image, return_tensors="pt", padding=True)
        
        # Move all input tensors to the same device as the model
        inputs = {k: v.to(device) for k, v in inputs.items()}
        
        # Run the model
        outputs = model(**inputs)
        image_embeddings = outputs.image_embeds
        text_embeddings = outputs.text_embeds
        
        # Compute similarity
        similarity_scores = F.cosine_similarity(image_embeddings, text_embeddings)
        
        # Get best match
        best_idx = similarity_scores.argmax().item()
        best_author = authors_list[best_idx]
        best_score = similarity_scores[best_idx].item()
        
        return best_author, best_score
    
    except Exception as e:
        print(f"Error processing {image_path}: {e}")
        return None, None

print("Elastic transform prediction function defined successfully!")

In [ ]:
# Load the results from CSV and filter for correct predictions
correct_predictions_df = pd.read_csv("top100_results_df.csv")
correct_predictions_df = correct_predictions_df[correct_predictions_df['correct'] == True]

# Convert to list of dictionaries for compatibility with existing code
correct_predictions = correct_predictions_df.to_dict('records')

print(f"Total correct predictions loaded from CSV: {len(correct_predictions)}")
print(f"\nFirst 5 correct predictions:")
for i, pred in enumerate(correct_predictions[:5], 1):
    print(f"{i}. {pred['title']} by {pred['true_author']} (confidence: {pred['confidence']:.4f})")

In [ ]:
# Re-predict authors on elastic-transformed versions of correctly predicted images
elastic_results = []

print(f"Re-predicting authors on {len(correct_predictions)} elastic-transformed images...\n")
print(f"Using ElasticTransform with alpha=250.0, sigma=5.0")
print("="*100)

total = len(correct_predictions)
for counter, result in enumerate(correct_predictions, 1):
    
    image_file = result['image_file']
    true_author = result['true_author']
    original_prediction = result['predicted_author']
    original_confidence = result['confidence']
    title = result['title']
    image_path = os.path.join("Images", image_file)
    
    # Show progress every 200 paintings or for first 10
    if counter <= 10 or counter % 200 == 0:
        print(f"\n[{counter}/{total}] Processing: {title} by {true_author}")
    
    # Predict author on elastic-transformed image
    predicted_author, confidence = predict_author_for_elastic_image(
        image_path, top_100_authors, model, processor, device, alpha=250.0, sigma=5.0
    )
    
    if predicted_author:
        is_correct = predicted_author == true_author
        symbol = "✅" if is_correct else "❌"
        
        # Only show detailed predictions for first 10 or every 200th painting
        if counter <= 10 or counter % 200 == 0:
            print(f"   Original (no transform): ✅ {original_prediction} (confidence: {original_confidence:.4f})")
            print(f"   Elastic transform: {symbol} {predicted_author} (confidence: {confidence:.4f})")
        
        elastic_results.append({
            'image_file': image_file,
            'title': title,
            'true_author': true_author,
            'original_prediction': original_prediction,
            'original_confidence': original_confidence,
            'elastic_prediction': predicted_author,
            'elastic_confidence': confidence,
            'still_correct': is_correct
        })

print(f"\n\nElastic transform re-prediction complete!")

In [ ]:
# Comprehensive comparison: Original vs Elastic Transform
if elastic_results:
    elastic_results_df = pd.DataFrame(elastic_results)
    
    print("\n" + "="*100)
    print("ELASTIC TRANSFORM vs ORIGINAL PREDICTION COMPARISON")
    print("="*100)
    
    # Calculate accuracy on elastic-transformed images
    elastic_accuracy = elastic_results_df['still_correct'].sum() / len(elastic_results_df) * 100
    
    print(f"\nOriginal Images (no transformation):")
    print(f"  Total paintings tested: {len(elastic_results_df)}")
    print(f"  All were correctly predicted: 100% (these were the correct predictions from original test)")
    
    print(f"\nElastic-Transformed Images (alpha=250.0, sigma=5.0):")
    print(f"  Total paintings tested: {len(elastic_results_df)}")
    print(f"  Still correctly predicted: {elastic_results_df['still_correct'].sum()}")
    print(f"  Accuracy: {elastic_accuracy:.2f}%")
    
    # Calculate drop in accuracy
    accuracy_drop = 100 - elastic_accuracy
    print(f"\nAccuracy Drop: {accuracy_drop:.2f}%")
    print(f"  ({elastic_results_df['still_correct'].sum()}/{len(elastic_results_df)} paintings remained correct after elastic transform)")
    
    # Confidence comparison
    print(f"\nConfidence Statistics:")
    print(f"  Original (no transform) - Average: {elastic_results_df['original_confidence'].mean():.4f}")
    print(f"  Elastic transform - Average: {elastic_results_df['elastic_confidence'].mean():.4f}")
    
    confidence_diff = elastic_results_df['elastic_confidence'].mean() - elastic_results_df['original_confidence'].mean()
    print(f"  Confidence change: {confidence_diff:+.4f}")
    
    # Compare with grayscale results if available
    if 'grayscale_results_df' in globals():
        print(f"\n\nComparison with Grayscale Transformation:")
        print(f"  Grayscale accuracy: {(grayscale_results_df['still_correct'].sum() / len(grayscale_results_df) * 100):.2f}%")
        print(f"  Elastic accuracy: {elastic_accuracy:.2f}%")
        elastic_vs_gray = elastic_accuracy - (grayscale_results_df['still_correct'].sum() / len(grayscale_results_df) * 100)
        print(f"  Elastic is {elastic_vs_gray:+.2f}% {'better' if elastic_vs_gray > 0 else 'worse'} than grayscale")
    
    # Analyze paintings that became incorrect
    became_incorrect_elastic = elastic_results_df[~elastic_results_df['still_correct']]
    
    if len(became_incorrect_elastic) > 0:
        print(f"\n\nPaintings that became INCORRECT after elastic transform ({len(became_incorrect_elastic)} total):")
        print("-"*100)
        
        print(f"\nMost common new predictions (what model confused them with):")
        new_predictions_elastic = became_incorrect_elastic['elastic_prediction'].value_counts().head(10)
        for author, count in new_predictions_elastic.items():
            print(f"  {author}: {count} paintings")
        
        print(f"\nAuthors most affected by elastic transform (lost correct predictions):")
        affected_authors_elastic = became_incorrect_elastic['true_author'].value_counts().head(10)
        for author, count in affected_authors_elastic.items():
            total_by_author = len(elastic_results_df[elastic_results_df['true_author'] == author])
            print(f"  {author}: {count}/{total_by_author} became incorrect ({count/total_by_author*100:.1f}%)")
        
        print(f"\nSample of paintings that lost correct prediction:")
        display(became_incorrect_elastic[['title', 'true_author', 'original_prediction', 'elastic_prediction', 
                                           'original_confidence', 'elastic_confidence']].head(20))
    
    # Paintings that remained correct
    remained_correct_elastic = elastic_results_df[elastic_results_df['still_correct']]
    
    if len(remained_correct_elastic) > 0:
        print(f"\n\nPaintings that remained CORRECT after elastic transform ({len(remained_correct_elastic)} total):")
        print("-"*100)
        
        print(f"\nAuthors most robust to elastic transform:")
        robust_authors_elastic = remained_correct_elastic['true_author'].value_counts().head(10)
        for author, count in robust_authors_elastic.items():
            total_by_author = len(elastic_results_df[elastic_results_df['true_author'] == author])
            print(f"  {author}: {count}/{total_by_author} remained correct ({count/total_by_author*100:.1f}%)")
        
        print(f"\nSample of paintings that remained correct:")
        display(remained_correct_elastic[['title', 'true_author', 'original_confidence', 'elastic_confidence']].head(20))
    
    # Side-by-side comparison of transformations
    if 'grayscale_results_df' in globals():
        print(f"\n\nSide-by-Side Comparison: Grayscale vs Elastic Transform")
        print("="*100)
        
        # Find paintings that failed in both
        gray_failed = set(grayscale_results_df[~grayscale_results_df['still_correct']]['image_file'])
        elastic_failed = set(elastic_results_df[~elastic_results_df['still_correct']]['image_file'])
        
        both_failed = gray_failed & elastic_failed
        only_gray_failed = gray_failed - elastic_failed
        only_elastic_failed = elastic_failed - gray_failed
        
        print(f"\nFailure patterns:")
        print(f"  Failed in BOTH transformations: {len(both_failed)} paintings")
        print(f"  Failed ONLY in grayscale: {len(only_gray_failed)} paintings")
        print(f"  Failed ONLY in elastic: {len(only_elastic_failed)} paintings")
        print(f"\nInterpretation:")
        print(f"  - Paintings that fail in both are likely highly sensitive to any distortion")
        print(f"  - Paintings that fail only in grayscale rely heavily on color information")
        print(f"  - Paintings that fail only in elastic are sensitive to geometric distortions but not color")
    
    print("\n\nAll Results (first 50 rows):")
    display(elastic_results_df[['title', 'true_author', 'original_prediction', 'elastic_prediction', 
                                 'original_confidence', 'elastic_confidence', 'still_correct']].head(50))

else:
    print("No results to display.")

In [ ]:
grayscale_results_df.to_csv("grayscale_results_df.csv", index=False)
perspective_results_df.to_csv("perspective_results_df.csv", index=False)
elastic_results_df.to_csv("elastic_results_df.csv", index=False)